In [3]:
from skorkart import scorecard_data

# Tüm hisseler tek Excel dosyasında, her biri ayrı sheet olarak:
results = scorecard_data(
    stock_codes=["THYAO", "GARAN"],
    start_period="2019/12",
    end_period="2025/03",
    lang="tr",
    save_to_excel=False,
    merge_to_single_excel=True
)

# Kodda DataFrame'lere doğrudan hisse adıyla erişebilirsiniz:
thy_financials = results["Financials"]["THYAO"]
garan_financials = results["Financials"]["GARAN"]

# Tüm finansalları dolaşmak için:
for hisse_adi, df in results["Financials"].items():
    print(f"{hisse_adi} finansalları:", df.head())

İşlem tamamlandı: THYAO
İşlem tamamlandı: GARAN
Tüm işlemler başarıyla tamamlandı.
THYAO finansalları:      Tarih  Net Satışlar Çeyrek (Mln TL)  \
0  2025/03                      176712.0   
1  2024/12                      193502.0   
2  2024/09                      221815.0   
3  2024/06                      182875.0   
4  2024/03                      147238.0   

   Net Satışlar (Yıllıklandırılmış) (Mln TL)  FAVÖK (Çeyrek) (Mln TL)  \
0                                   774904.0                  10167.0   
1                                   745430.0                  27603.0   
2                                   697545.0                  59088.0   
3                                   644821.0                  34065.0   
4                                   569579.0                  15500.0   

   FAVÖK (Yıllıklandırılmış) (Mln TL)  \
0                            130923.0   
1                            136256.0   
2                            130725.0   
3                            

In [3]:
from isyatirimhisse import fetch_stock_data, fetch_index_data, fetch_financials
import pandas as pd

df = fetch_financials(
    symbols="THYAO",
    start_year=2025,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)
df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
list(df_pivoted.columns)

['symbol',
 'period',
 '(subtotal)',
 'adjustments_to_share_capital',
 'adjustments:',
 'assets_to_be_sold',
 'current_assets',
 'capital_expenditures_(capex)',
 'cash_net_inc._/_dec._before_for.cur._trans._dif.',
 'cash_at_the_end_of_the_quarter',
 'cash_at_the_beginning_of_the_quarter',
 'cash_from_core_operations_(+)',
 'cash_from_financial_operations',
 'cash_from_investment_operations',
 'cash_from_other_financial_operations',
 'cash_from_other_investment_operations',
 'cash_from_other_operations',
 'cash\xa0and\xa0cash\xa0equivalents',
 'change_in_cash_&_cash_equivalents',
 'change_in_financial_debt',
 'change_in_provisions',
 'change_in_working_capital',
 'continuing_operations',
 'cost_of_sales',
 'current_year_income_/(losses)',
 'discontinued_operations',
 'deferred_tax_liabilities',
 'depreciation_&_amortisation',
 'depreciation_&_amortization',
 'diluted_earnings_per_share',
 'distrubition_of_profit_(loss)',
 'dividends_paid',
 'domestic_sales',
 'earnings_before_adjustment

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from isyatirimhisse import fetch_financials

conn = sqlite3.connect("C:/Users/KULLANICI/Desktop/portfolio-backtest-project/data/database.db")

ticker_dict = {"THYAO":1,"BIMAS":2}

df = fetch_financials(
    symbols=list(ticker_dict.keys()),
    start_year=2025,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)

df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
df_pivoted = df_pivoted.rename(columns={'net_sales': 'revenue',"gross_profit_(loss)": "gross_profit","operating_profits":"operating_profit","net_profit_after_taxes":"net_income","long_term_liabilities":"long_term_debt","short_term_liabilities":"short_term_debt","shareholders_equity":"equity"})
df_pivoted['ebitda'] = df_pivoted['operating_profit'] + df_pivoted['depreciation_&_amortization']
df_pivoted['fixed_assets'] = df_pivoted['total_assets'] - df_pivoted['current_assets']
df_pivoted['gross_debt'] = df_pivoted['short-term_financial_loans'] + df_pivoted['long-term_financial_loans']
df_pivoted['net_debt'] = df_pivoted['gross_debt'] - df_pivoted["cash\xa0and\xa0cash\xa0equivalents"]
df_pivoted['company_id'] = df_pivoted['symbol'].map(ticker_dict)
df_pivoted["date_of_publish"] = np.nan

df_pivoted = df_pivoted[['company_id',"period","date_of_publish","revenue","gross_profit","operating_profit","ebitda","net_income","current_assets","fixed_assets","long_term_debt","short_term_debt","gross_debt","net_debt","equity"]]

df_pivoted.to_sql('financial', conn, if_exists='append', index=False)
conn.close()


In [2]:
df_pivoted

,company_id,period,date_of_publish,revenue,gross_profit,operating_profit,ebitda,net_income,current_assets,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity
0,2,2025/3,NaN,1.477227e+11,2.585878e+10,-1.297669e+09,4.883661e+09,2.710405e+09,1.052041e+11,1.815128e+11,4.924420e+10,1.044967e+11,4.209025e+10,3.413588e+10,1.329760e+11
1,2,2025/6,NaN,3.098349e+11,5.889611e+10,2.401768e+09,1.471529e+10,5.553111e+09,9.922073e+10,1.962433e+11,5.409232e+10,1.046679e+11,4.646131e+10,4.354015e+10,1.367039e+11
2,1,2025/3,NaN,1.767120e+11,1.173100e+10,-2.879000e+09,1.753400e+10,-1.854000e+09,3.526760e+11,1.182270e+12,4.244820e+11,3.930280e+11,5.376700e+11,4.488490e+11,7.174360e+11
3,1,2025/6,NaN,4.080360e+11,5.614600e+10,2.452900e+10,6.748200e+10,2.493500e+10,3.970760e+11,1.255513e+12,4.688030e+11,4.316660e+11,5.845920e+11,4.858010e+11,7.521200e+11


# ÇEYREKLİK VERİ YAPMA - 1 (YAVAŞ YÖNTEM)

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from isyatirimhisse import fetch_financials

ticker_dict = {"THYAO":2,"BIMAS":1}

df = fetch_financials(
    symbols=list(ticker_dict.keys()),
    start_year=2024,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)

df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
df_pivoted = df_pivoted.rename(columns={'net_sales': 'revenue',"gross_profit_(loss)": "gross_profit","operating_profits":"operating_profit","net_profit_after_taxes":"net_income","long_term_liabilities":"long_term_debt","short_term_liabilities":"short_term_debt","shareholders_equity":"equity"})
df_pivoted['ebitda'] = (df_pivoted['gross_profit'] - 
               df_pivoted['marketing_selling_&_distrib._expenses_(-)'].abs() -
               df_pivoted['general_administrative_expenses_(-)'].abs() - 
               df_pivoted['research_&_development_expenses_(-)'].abs() +
               df_pivoted['depreciation_&_amortization'])
df_pivoted['fixed_assets'] = df_pivoted['total_assets'] - df_pivoted['current_assets']
df_pivoted['gross_debt'] = (
    df_pivoted['short-term_financial_loans'] + 
    df_pivoted['long-term_financial_loans'] + 
    df_pivoted['other_short-term_financial_liabilities'] +
    df_pivoted['short-term_loans_from_financial_operations'] + 
    df_pivoted['long-term_loans_from_financial_operations']
)
df_pivoted['net_debt'] = df_pivoted['gross_debt'] - df_pivoted["cash\xa0and\xa0cash\xa0equivalents"] - df_pivoted['short-term_financial_investments']
df_pivoted['company_id'] = df_pivoted['symbol'].map(ticker_dict)
df_pivoted["date_of_publish"] = np.nan
df_pivoted[["period_year","period_month"]] = df_pivoted["period"].str.split("/",expand=True)
df_pivoted['period_month'] = pd.to_numeric(df_pivoted['period_month'])
df_pivoted['period_year'] = pd.to_numeric(df_pivoted['period_year'])


def quarter_financials(df):

    kalemler = ["revenue","gross_profit","operating_profit","ebitda","net_income"]

    df = df.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)

    for kalem in kalemler:
        kalem_q = kalem + "_q" 
        for row_idx in range(len(df)): 

            if df.iloc[row_idx]["period_month"] == 3:
                df.at[row_idx,kalem_q] = df.iloc[row_idx][kalem]
            elif df.iloc[row_idx]["period_month"] in [6,9,12]:
                try:
                    prev_row = df[
                        (df["company_id"] == df.iloc[row_idx]["company_id"]) &
                        (df["period_year"] == df.iloc[row_idx]["period_year"]) &
                        (df["period_month"] == df.iloc[row_idx]["period_month"] - 3)
                    ]
                    df.at[row_idx,kalem_q] = df.iloc[row_idx][kalem] - prev_row.iloc[0][kalem]
                except IndexError:
                    df.at[row_idx,kalem_q] = np.nan
            else:
                df.at[row_idx,kalem_q] = np.nan

    return df

df_pivoted = quarter_financials(df_pivoted)


df_pivoted = df_pivoted[['company_id',"period_year","period_month","date_of_publish","revenue_q","gross_profit_q","operating_profit_q","ebitda_q","net_income_q","current_assets","fixed_assets","long_term_debt","short_term_debt","gross_debt","net_debt","equity"]]

df_pivoted

    

,company_id,period_year,period_month,date_of_publish,revenue_q,gross_profit_q,operating_profit_q,ebitda_q,net_income_q,current_assets,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity
0,1,2024,3,NaN,1.433081e+11,2.465973e+10,1.658360e+09,6.144589e+09,5.356072e+09,7.645432e+10,1.125850e+11,3.041031e+10,7.445325e+10,2.698775e+10,1.754948e+10,8.417579e+10
1,1,2024,6,NaN,1.561526e+11,2.621189e+10,2.904710e+08,5.307584e+09,6.431079e+09,7.409616e+10,1.264848e+11,3.264443e+10,7.856430e+10,3.065833e+10,2.596097e+10,8.937220e+10
2,1,2024,9,NaN,6.799874e+10,1.247433e+10,5.211530e+08,3.218732e+09,2.093536e+09,8.366406e+10,1.402811e+11,3.601439e+10,8.625010e+10,3.391270e+10,1.444870e+10,1.016807e+11
3,1,2024,12,NaN,2.387393e+11,4.261396e+10,3.715749e+09,1.164827e+10,7.835769e+09,8.638702e+10,1.887871e+11,4.974624e+10,8.712409e+10,4.314134e+10,3.352781e+10,1.383038e+11
4,1,2025,3,NaN,1.477227e+11,2.585878e+10,-1.297669e+09,5.282371e+09,2.710405e+09,1.052041e+11,1.815128e+11,4.924420e+10,1.044967e+11,4.209025e+10,1.860062e+10,1.329760e+11
5,1,2025,6,NaN,1.621122e+11,3.303733e+10,3.699437e+09,9.740773e+09,2.842706e+09,9.922073e+10,1.962433e+11,5.409232e+10,1.046679e+11,4.646131e+10,3.283389e+10,1.367039e+11
6,2,2024,3,NaN,1.472380e+11,1.614000e+10,1.203000e+09,1.550000e+10,6.918000e+09,2.755980e+11,8.903780e+11,3.438810e+11,3.065740e+11,4.314230e+11,2.484780e+11,5.155210e+11
7,2,2024,6,NaN,1.828750e+11,3.524600e+10,1.910400e+10,3.406500e+10,3.039100e+10,3.120300e+11,9.218130e+11,3.444830e+11,3.315930e+11,4.255880e+11,2.352110e+11,5.577670e+11
8,2,2024,9,NaN,2.218150e+11,6.093900e+10,4.504800e+10,5.908800e+10,5.152200e+10,3.388820e+11,1.014787e+12,3.869560e+11,3.480220e+11,4.949635e+11,2.821945e+11,6.186910e+11
9,2,2024,12,NaN,3.177945e+11,5.441138e+10,2.844267e+10,5.032223e+10,4.342707e+10,3.989198e+11,1.234055e+12,4.434321e+11,3.961465e+11,5.716311e+11,3.219232e+11,7.933967e+11


# shift(), diff() ve groupby() öğreneyim

In [24]:
import pandas as pd
import numpy as np

# Örnek kümülatif gelir verisi
data = {
    'Şirket': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
    'Yıl': [2023, 2023, 2023, 2023, 2024, 2024, 2024, 2024],
    'Çeyrek': [1, 2, 3, 4, 1, 2, 3, 4],
    'Kümülatif Gelir': [100, 250, 450, 700, 150, 350, 600, 900]
}

df = pd.DataFrame(data)
df

,Şirket,Yıl,Çeyrek,Kümülatif Gelir
0,A,2023,1,100
1,A,2023,2,250
2,A,2023,3,450
3,A,2023,4,700
4,B,2024,1,150
5,B,2024,2,350
6,B,2024,3,600
7,B,2024,4,900


In [ ]:
df["Çeyreklik Gelir"] = df["Kümülatif Gelir"] - df["Kümülatif Gelir"].shift(1)
df

,Şirket,Yıl,Çeyrek,Kümülatif Gelir,Çeyreklik Gelir
0,A,2023,1,100,NaN
1,A,2023,2,250,150.0
2,A,2023,3,450,200.0
3,A,2023,4,700,250.0
4,B,2024,1,150,-550.0
5,B,2024,2,350,200.0
6,B,2024,3,600,250.0
7,B,2024,4,900,300.0


In [29]:
# Her bir şirket grubu içinde kümülatif geliri bir satır kaydır
df['Önceki Kümülatif Gelir'] = df.groupby('Şirket')['Kümülatif Gelir'].shift(1)

# Çeyreklik geliri hesapla
df['Çeyreklik Gelir'] = df['Kümülatif Gelir'] - df['Önceki Kümülatif Gelir']

df

,Şirket,Yıl,Çeyrek,Kümülatif Gelir,Önceki Kümülatif Gelir,Çeyreklik Gelir
0,A,2023,1,100,NaN,NaN
1,A,2023,2,250,100.0,150.0
2,A,2023,3,450,250.0,200.0
3,A,2023,4,700,450.0,250.0
4,B,2024,1,150,NaN,NaN
5,B,2024,2,350,150.0,200.0
6,B,2024,3,600,350.0,250.0
7,B,2024,4,900,600.0,300.0


In [32]:
import pandas as pd
import numpy as np

# Örnek kümülatif gelir verisi
data = {
    'Şirket': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
    'Yıl': [2023, 2023, 2023, 2023, 2024, 2024, 2024, 2024],
    'Çeyrek': [1, 2, 3, 4, 1, 2, 3, 4],
    'Kümülatif Gelir': [100, 250, 450, 700, 150, 350, 600, 900]
}

df = pd.DataFrame(data)

df["Çeyreklik Gelir"] = df.groupby("Şirket")["Kümülatif Gelir"].diff()
df

,Şirket,Yıl,Çeyrek,Kümülatif Gelir,Çeyreklik Gelir
0,A,2023,1,100,NaN
1,A,2023,2,250,150.0
2,A,2023,3,450,200.0
3,A,2023,4,700,250.0
4,B,2024,1,150,NaN
5,B,2024,2,350,200.0
6,B,2024,3,600,250.0
7,B,2024,4,900,300.0


In [ ]:
import pandas as pd
import numpy as np

# Örnek kümülatif gelir verisi
data = {
    'Şirket': ['A', 'A', 'A', 'A','A', 'A', 'A', 'A', 'B', 'B', 'B', 'B'],
    'Yıl': [2023, 2023, 2023, 2023, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024],
    'Çeyrek': [1, 2, 3, 4,1, 2, 3, 4, 1, 2, 3, 4],
    'Kümülatif Gelir': [100, 250, 450, 700, 200,300,350,700, 150, 350, 600, 900]
}

df = df.sort_values(by=['Şirket', 'Yıl', 'Çeyrek']).reset_index(drop=True)

df['Çeyreklik Gelir'] = np.where(
    df['Çeyrek'] == 1,
    df['Kümülatif Gelir'],
    df.groupby(['Şirket',"Yıl"])['Kümülatif Gelir'].diff()
)

df

,Şirket,Yıl,Çeyrek,Kümülatif Gelir,Çeyreklik Gelir
0,A,2023,2,250,NaN
1,A,2023,3,450,200.0
2,A,2023,4,700,250.0
3,A,2024,1,200,200.0
4,A,2024,2,300,100.0
5,A,2024,3,350,50.0
6,A,2024,4,700,350.0
7,B,2024,1,150,150.0
8,B,2024,2,350,200.0
9,B,2024,3,600,250.0


# Çeyreklik veri yapma - 2 (vectorized işlemlerle)

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from isyatirimhisse import fetch_financials

ticker_dict = {"THYAO":2,"BIMAS":1}

df = fetch_financials(
    symbols=list(ticker_dict.keys()),
    start_year=2023,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)

df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
df_pivoted = df_pivoted.rename(columns={'net_sales': 'revenue',"gross_profit_(loss)": "gross_profit","operating_profits":"operating_profit","net_profit_after_taxes":"net_income","long_term_liabilities":"long_term_debt","short_term_liabilities":"short_term_debt","shareholders_equity":"equity","cash\xa0and\xa0cash\xa0equivalents":"cash_and_cash_equivalents"})
df_pivoted['ebitda'] = (df_pivoted['gross_profit'] - 
               df_pivoted['marketing_selling_&_distrib._expenses_(-)'].abs() -
               df_pivoted['general_administrative_expenses_(-)'].abs() - 
               df_pivoted['research_&_development_expenses_(-)'].abs() +
               df_pivoted['depreciation_&_amortization'])
df_pivoted['fixed_assets'] = df_pivoted['total_assets'] - df_pivoted['current_assets']
df_pivoted['gross_debt'] = (
    df_pivoted['short-term_financial_loans'] + 
    df_pivoted['long-term_financial_loans'] + 
    df_pivoted['other_short-term_financial_liabilities'] +
    df_pivoted['short-term_loans_from_financial_operations'] + 
    df_pivoted['long-term_loans_from_financial_operations']
)
df_pivoted['net_debt'] = df_pivoted['gross_debt'] - df_pivoted["cash_and_cash_equivalents"] - df_pivoted['short-term_financial_investments']
df_pivoted['company_id'] = df_pivoted['symbol'].map(ticker_dict)
df_pivoted["date_of_publish"] = np.nan
df_pivoted[["period_year","period_month"]] = df_pivoted["period"].str.split("/",expand=True)
df_pivoted['period_month'] = pd.to_numeric(df_pivoted['period_month'])
df_pivoted['period_year'] = pd.to_numeric(df_pivoted['period_year'])


df_pivoted = df_pivoted.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
kalemler = ["revenue","gross_profit","operating_profit","ebitda","net_income"]

for kalem in kalemler:
    kalem_q = kalem + "_q"
    kalem_ttm = kalem + "_ttm"


    df_pivoted[kalem_q] = np.where(
        df_pivoted['period_month'] == 3,
        df_pivoted[kalem],
        df_pivoted.groupby(['company_id',"period_year"])[kalem].diff()
    )

    df_pivoted[kalem_ttm] = df_pivoted.groupby('company_id')[kalem_q].rolling(window=4).sum().values


df_pivoted = df_pivoted[['company_id',"period_year","period_month","date_of_publish","revenue_ttm","gross_profit_ttm","operating_profit_ttm","ebitda_ttm","net_income_ttm","revenue_q","gross_profit_q","operating_profit_q","ebitda_q","net_income_q","cash_and_cash_equivalents","current_assets","fixed_assets","long_term_debt","short_term_debt","gross_debt","net_debt","equity"]]

df_pivoted


,company_id,period_year,period_month,date_of_publish,revenue_ttm,gross_profit_ttm,operating_profit_ttm,ebitda_ttm,net_income_ttm,revenue_q,...,operating_profit_q,ebitda_q,net_income_q,current_assets,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity
0,1,2023,3,NaN,NaN,NaN,NaN,NaN,NaN,9.607751e+10,...,1.100200e+07,3.266134e+09,1.250558e+09,3.597851e+10,4.005171e+10,1.250529e+10,3.719496e+10,1.454609e+10,1.051510e+10,2.632998e+10
1,1,2023,6,NaN,NaN,NaN,NaN,NaN,NaN,1.015005e+11,...,1.492079e+09,4.603347e+09,3.982056e+09,3.934852e+10,4.430060e+10,1.505714e+10,4.244076e+10,1.709096e+10,1.054108e+10,2.615122e+10
2,1,2023,9,NaN,NaN,NaN,NaN,NaN,NaN,1.333126e+11,...,4.622582e+09,8.586810e+09,8.058753e+09,5.166231e+10,4.776539e+10,1.646180e+10,5.292282e+10,1.918579e+10,8.850537e+09,3.004308e+10
3,1,2023,12,NaN,4.742004e+11,7.461878e+10,5.717514e+09,1.985372e+10,2.229972e+10,1.433098e+11,...,-4.081490e+08,3.397430e+09,9.008352e+09,7.862717e+10,1.352450e+11,3.510860e+10,7.807136e+10,3.225127e+10,2.216161e+10,1.006923e+11
4,1,2024,3,NaN,5.214310e+11,8.464737e+10,7.364872e+09,2.273218e+10,2.640523e+10,1.433081e+11,...,1.658360e+09,6.144589e+09,5.356072e+09,7.645432e+10,1.125850e+11,3.041031e+10,7.445325e+10,2.698775e+10,1.754948e+10,8.417579e+10
5,1,2024,6,NaN,5.760832e+11,9.494128e+10,6.163264e+09,2.343641e+10,2.885426e+10,1.561526e+11,...,2.904710e+08,5.307584e+09,6.431079e+09,7.409616e+10,1.264848e+11,3.264443e+10,7.856430e+10,3.065833e+10,2.596097e+10,8.937220e+10
6,1,2024,9,NaN,5.107693e+11,8.298674e+10,2.061835e+09,1.806834e+10,2.288904e+10,6.799874e+10,...,5.211530e+08,3.218732e+09,2.093536e+09,8.366406e+10,1.402811e+11,3.601439e+10,8.625010e+10,3.391270e+10,1.444870e+10,1.016807e+11
7,1,2024,12,NaN,6.061988e+11,1.059599e+11,6.185733e+09,2.631918e+10,2.171646e+10,2.387393e+11,...,3.715749e+09,1.164827e+10,7.835769e+09,8.638702e+10,1.887871e+11,4.974624e+10,8.712409e+10,4.314134e+10,3.352781e+10,1.383038e+11
8,1,2025,3,NaN,6.106133e+11,1.071590e+11,3.229704e+09,2.545696e+10,1.907079e+10,1.477227e+11,...,-1.297669e+09,5.282371e+09,2.710405e+09,1.052041e+11,1.815128e+11,4.924420e+10,1.044967e+11,4.209025e+10,1.860062e+10,1.329760e+11
9,1,2025,6,NaN,6.165729e+11,1.139844e+11,6.638670e+09,2.989015e+10,1.548242e+10,1.621122e+11,...,3.699437e+09,9.740773e+09,2.842706e+09,9.922073e+10,1.962433e+11,5.409232e+10,1.046679e+11,4.646131e+10,3.283389e+10,1.367039e+11


In [4]:
import pandas as pd
import numpy as np
import sqlite3
from isyatirimhisse import fetch_financials

ticker_dict = {"THYAO":2,"BIMAS":1}

df = fetch_financials(
    symbols=list(ticker_dict.keys()),
    start_year=2023,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)

df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
df_pivoted = df_pivoted.rename(columns={'net_sales': 'revenue',"gross_profit_(loss)": "gross_profit","operating_profits":"operating_profit","net_profit_after_taxes":"net_income","long_term_liabilities":"long_term_debt","short_term_liabilities":"short_term_debt","shareholders_equity":"equity","cash\xa0and\xa0cash\xa0equivalents":"cash_and_cash_equivalents"})
df_pivoted['ebitda'] = (df_pivoted['gross_profit'] - 
               df_pivoted['marketing_selling_&_distrib._expenses_(-)'].abs() -
               df_pivoted['general_administrative_expenses_(-)'].abs() - 
               df_pivoted['research_&_development_expenses_(-)'].abs() +
               df_pivoted['depreciation_&_amortization'])
df_pivoted['fixed_assets'] = df_pivoted['total_assets'] - df_pivoted['current_assets']
df_pivoted['gross_debt'] = (
    df_pivoted['short-term_financial_loans'] + 
    df_pivoted['long-term_financial_loans'] + 
    df_pivoted['other_short-term_financial_liabilities'] +
    df_pivoted['short-term_loans_from_financial_operations'] + 
    df_pivoted['long-term_loans_from_financial_operations']
)
df_pivoted['net_debt'] = df_pivoted['gross_debt'] - df_pivoted["cash_and_cash_equivalents"] - df_pivoted['short-term_financial_investments']
df_pivoted['company_id'] = df_pivoted['symbol'].map(ticker_dict)
df_pivoted["date_of_publish"] = np.nan
df_pivoted[["period_year","period_month"]] = df_pivoted["period"].str.split("/",expand=True)
df_pivoted['period_month'] = pd.to_numeric(df_pivoted['period_month'])
df_pivoted['period_year'] = pd.to_numeric(df_pivoted['period_year'])


df_pivoted = df_pivoted.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
kalemler = ["revenue","gross_profit","operating_profit","ebitda","net_income"]

for kalem in kalemler:
    kalem_q = kalem + "_q"
    kalem_ttm = kalem + "_ttm"


    df_pivoted[kalem_q] = np.where(
        df_pivoted['period_month'] == 3,
        df_pivoted[kalem],
        df_pivoted.groupby(['company_id',"period_year"])[kalem].diff()
    )

    df_pivoted[kalem_ttm] = df_pivoted.groupby('company_id')[kalem_q].rolling(window=4).sum().values

list(df_pivoted.columns.unique())

['symbol',
 'period',
 '(subtotal)',
 'adjustments_to_share_capital',
 'adjustments:',
 'assets_to_be_sold',
 'current_assets',
 'capital_expenditures_(capex)',
 'cash_net_inc._/_dec._before_for.cur._trans._dif.',
 'cash_at_the_end_of_the_quarter',
 'cash_at_the_beginning_of_the_quarter',
 'cash_from_core_operations_(+)',
 'cash_from_financial_operations',
 'cash_from_investment_operations',
 'cash_from_other_financial_operations',
 'cash_from_other_investment_operations',
 'cash_from_other_operations',
 'cash_and_cash_equivalents',
 'change_in_cash_&_cash_equivalents',
 'change_in_financial_debt',
 'change_in_provisions',
 'change_in_working_capital',
 'continuing_operations',
 'cost_of_sales',
 'current_year_income_/(losses)',
 'discontinued_operations',
 'deferred_tax_liabilities',
 'depreciation_&_amortisation',
 'depreciation_&_amortization',
 'diluted_earnings_per_share',
 'distrubition_of_profit_(loss)',
 'dividends_paid',
 'domestic_sales',
 'earnings_before_adjustments',
 'ear